# 🌐 ThaiCoNet : Thai Co-occurrence Network Analysis (Pipeline)

### 💡 About this notebook

**[Access Project Repository (Github)](https://github.com/ChotanansubSoph/ThNTA)**



🧑🏻‍💻 **Notebook Contributor**


*   Chotanansub Sophaken
*   Kantapong Vongpanich

**Department of Computer Eningeering, Engineering Faculty**

*King Mongkut’s University of Technology Thonburi (KMUTT)*

*Junior Science Talent Project and Siam Commercial Bank Scholarship (JSTP-SCB Scholarship)*

---



Approach reference:

* *A. Takhom, D. Leenoi, C. Sophaken, P. Boonkwan, and T. Supnithi, “An Approach of Network Analysis Enhancing Knowledge Extraction in Thai Newspapers Contexts,” J. Intell. Informatics Smart Technol., vol. 6, no. October 2021, pp. 19–24, 2021 [Acess](https://jiist.aiat.or.th/assets/uploads/1635853027829tBupD1635602106085fdegH39.pdf)*

* Sophaken, C., Vongpanich, K., Takhom, A., Boonkwan, P., & Supnithi, T. (2023). Unsupervised Detection of Domain Switching in Thai Multidisciplinary Online News. IIAI Letters on Informatics and Interdisciplinary Research, 3. [Access](https://iaiai.org/letters/index.php/liir/article/view/77/50)

🎓 Acknowledgement
* Dr. Akkharawoot TakhomDepartment of Electrical and Computer Engineering
Thammasat University

### ⚙️ Tools & Resorces Preparation

Library & Module Installation



* Run the code once.
* If you encounter any errors during the initial execution it may be due to factors such as dependencies or system configurations. To address any encountered errors, simply restart the runtime or kernel. Afterward, run the code cell again to ensure a successful execution.



In [1]:
import subprocess

def install_packages():
    subprocess.check_call(['pip', 'install', '--upgrade', 'pip'])
    subprocess.check_call(['pip', 'install', '--upgrade', 'setuptools', 'wheel'])
    subprocess.check_call(['pip', 'install', 'tltk==1.6.8', '-q'])
    subprocess.check_call(['pip', 'install', 'pythainlp==4.0.2', '-q'])
    subprocess.check_call(['pip', 'install', 'pyvis==0.1.9', '-q'])
    subprocess.check_call(['apt-get', 'install', '-y', 'graphviz', 'libgraphviz-dev', 'pkg-config', '-q'])
    subprocess.check_call(['pip', 'install', 'pygraphviz==1.7', '-q'])

install_packages()

Library preparation

In [2]:
#Data manipulation
import pandas as pd
import numpy as np

#NLP
import nltk
from nltk import FreqDist, bigrams
from operator import itemgetter
from nltk.tokenize import word_tokenize as term_tokenize
nltk.download('punkt')
import tltk
from pythainlp import word_tokenize,  pos_tag
from pythainlp.corpus.common import thai_stopwords as pythainlp_stopwords

#Graph Visulazation
import networkx as nx
import matplotlib.pyplot as plt
from pyvis.network import Network
from IPython.display import display, HTML

#Add-on
from tqdm.notebook import tqdm_notebook as tqdm
from operator import itemgetter
from collections import Counter,defaultdict
import re
import sys
import os
import requests

__is_ipython_kernel__ = lambda: True if 'ipykernel' in sys.modules and __name__ == "__main__" else False

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Sample Resources prepararion

In [3]:
#sample data
def download_data(url, file_name=None):
    if file_name is None:
        file_name = url.split('/')[-1]
    response = requests.get(url)
    with open(file_name, "wb") as file:
        file.write(response.content)

if __is_ipython_kernel__:
    url = "https://github.com/ChotanansubSoph/ThNTA/raw/main/resources/sample_data/thai_electronic_news_2022.csv"
    download_data(url=url)

### Data Preprocessing

In [4]:
########## String operation ##########
def isEnglish(s):
  return all(ord(char) < 128 for char in s)


########## List Manipulation ##########
def flatten_nested_list(nested_list):
  flattened_list = [item for sublist in nested_list for item in sublist]
  return flattened_list


######### DataFrame Manipulation #######
def convert_dataframe_to_paired_tuples(df):
    return list(zip(df.iloc[:, 0].tolist(), df.iloc[:, 1].tolist()))

In [5]:
########## Stopwords ##########
def read_stopwords(file_path : str) ->list:
  with open(file_path, 'r', encoding='utf-8') as file:
      lines = file.readlines()
  stopwords = [line.strip() for line in lines]
  return stopwords

########## Tokenization ##########

def tltk_tokenize_pos(text): #Primaly Tokenizer
  result = flatten_nested_list(tltk.nlp.pos_tag(text))
  return result


def pythainlp_tokenize_pos(text): #Secondary Tokenizer
  wordList= word_tokenize(text, keep_whitespace=False)
  posList = pos_tag(wordList,corpus='pud')
  return posList


def TNC_extract_tltk_pos_pairs(result): #Inactivated
    word_pos_pairs = []
    pattern = r'<w tran="(.*?)" POS="(.*?)">(.*?)</w>'
    matches = re.findall(pattern, result)

    for match in matches:
        word_pos_pairs.append((match[2], match[1]))

    return word_pos_pairs

def TNC_tokenize_pos_ner_(text): #Inactivated
  result = []
  for partial_text in text.split(" "):
    partial_text = partial_text.replace(")"," ").replace("("," ")
    result += tltk.nlp.TNC_tag(partial_text,POS="Y")
  return tltk.nlp.ner(TNC_extract_tltk_pos_pairs(result))


########## Term Frequency ##########
def count_word_frequency(pairs_data):
    words = [word for sublist in pairs_data for word in sublist]
    tokens = term_tokenize(" ".join(words))
    freq_dist = FreqDist(tokens)
    return freq_dist


def count_word_pos_frequency(data):
    flat_data = [item for sublist in data for item in sublist]
    word_pos_freq = Counter(flat_data)
    result = [(word, pos, freq) for (word, pos), freq in word_pos_freq.items()]
    result.sort(key=lambda x: x[2], reverse=True)
    return result

### Text preprocess

In [6]:
def text_tokenize(text: str,tokenizer="pythainlp") -> list:
    term_pairs = list()
    if tokenizer == "tltk":
      term_pairs = tltk_tokenize_pos(text)

    elif tokenizer == "pythainlp":
      term_pairs = pythainlp_tokenize_pos(text)


    return term_pairs


def pos_filter(pos_pairs: list, stopwords: set, keep_pos=['NOUN','VERB']):
  stopwords = []
  regex = re.compile('[@_!#$%^&*()<>?/\|}{~:.]')
  filtered_pairs = [(term,pos) for term, pos in pos_pairs
              if pos in keep_pos
              and term not in stopwords
              and len(term) > 1
              and not isEnglish(term)
              and regex.search(term) is None
              and "\xa0" not in term]

  return filtered_pairs


def feed_preprocess(docs: list, stopwords = None, tokenizer="pythainlp", keep_pos=None) -> list:
    preprocessed_docs = []
    if stopwords is None:
      stopwords = pythainlp_stopwords()

    for text in tqdm(docs):
        pos_pairs = text_tokenize(
                      text=text,
                      tokenizer=tokenizer,
                    )
        preprocessed_terms = pos_filter(pos_pairs=pos_pairs, stopwords=stopwords)
        preprocessed_docs.append(preprocessed_terms)

    return preprocessed_docs

Tokenization & Term Filtering

In [7]:
if __is_ipython_kernel__:
  news_df = pd.read_csv("thai_electronic_news_2022.csv")
  news_df

In [8]:
if __is_ipython_kernel__:
  tokenized_data = feed_preprocess(news_df["content"],tokenizer="pythainlp") #approximate time ~5 min

  0%|          | 0/8981 [00:00<?, ?it/s]

In [9]:
if __is_ipython_kernel__:
  tokenized_data[0][:10] , tokenized_data[1][:10]

In [10]:
if __is_ipython_kernel__:
  tokenized_freq = count_word_pos_frequency(tokenized_data)
  tokenized_freq[:15]

generate bag of co-occurence terminology

In [11]:
def generate_bigram_freq(term_list)->list:
    bigram_list = []

    for word_list in term_list:
        try:
            bigrams_list = list(bigrams(word_list))
            bigram_list.extend(bigrams_list)
        except:
            continue

    frequency_dist = FreqDist(bigram_list)
    bigram_freq = sorted(frequency_dist.items(), key=itemgetter(1), reverse=True)

    return bigram_freq


def generate_trigrams(data):
  result = []
  terms = flatten_nested_list(data)
  for i in range(len(terms[:-2])):
    triple = (terms[i],terms[i+1],terms[i+2])
    result.append(triple)
  return result


def count_triple_frequency(triples):
    triple_frequency = Counter(triples)
    result = list(triple_frequency.items())
    return  sorted(result, key=lambda x: x[1], reverse=True)

In [12]:
if __is_ipython_kernel__:
  cooc_data = generate_trigrams(tokenized_data)
  cooc_data[:10]

In [13]:
if __is_ipython_kernel__:
  cooc_freqs = count_triple_frequency(cooc_data)
  cooc_freqs[:10]

In [14]:
def filter_pos_triples(data,keeps = [('NOUN','VERB','NOUN')]):
    filtered_triples = [ pair
        for pair in data
        if pair[0][0][1] == 'NOUN' and pair[0][1][1] == 'VERB' and pair[0][2][1] == 'NOUN'
    ]
    return filtered_triples

In [15]:
if __is_ipython_kernel__:
  filtered_cooc = filter_pos_triples(cooc_freqs)
  filtered_cooc[:20]

In [16]:
def bgs_filter_extreme(bgs_list, min_percent=0.05, max_percent=0.8):
  result = list()
  bgs_list = sorted(bgs_list, key=itemgetter(1), reverse=True)
  most_freq = bgs_list[0][1]
  max_freq = most_freq * max_percent
  min_freq = most_freq * min_percent

  result = [(pair, count) for pair, count in bgs_list if min_freq <= count <= max_freq and pair[0] != pair[1]]
  return result


## Visualization

In [17]:
def visualize_cooccurrence(data, file_name="thaiconet_result.html"):
    # Phase 1: NetworkX
    G = nx.Graph()

    # Create a dictionary to store the degree of each node
    node_degrees = {}

    for triples, freq in data:
        sbj = triples[0][0]
        pred = triples[1][0]
        obj = triples[2][0]

        # Update the degree of term1
        node_degrees[sbj] = node_degrees.get(sbj, 0) + 1

        # Update the degree of term2
        node_degrees[obj] = node_degrees.get(obj, 0) + 1

        # Add nodes and set their size based on the degree
        G.add_node(sbj, size=min(node_degrees[sbj] * 7, 80))
        G.add_node(obj, size=min(node_degrees[obj] * 7, 80))

        # Add edge
        G.add_edge(sbj, obj, weight=freq, label=pred)

    pos = nx.spring_layout(G)
    sizes = [G.nodes[n]['size'] for n in G.nodes()]

    # Phase 2: PyviZ
    net = Network(height="800px", width="100%", notebook=True)

    for node in G.nodes():
        net.add_node(node, size=G.nodes[node]['size'])

    for u, v, data in G.edges(data=True):
        weight = data['weight']
        label = data['label']
        color = "orange" if weight > 45 else "gray"
        net.add_edge(u, v, value=weight, color=color, label=label)

    net.show(file_name)

In [18]:
if __is_ipython_kernel__:
  visualize_cooccurrence(filtered_cooc[100:200])
  display(HTML("thaiconet_result.html"))